In [ ]:



# 如何实现对某网站内容作为知识库，实现问答机器人




In [24]:
from langchain_community.document_loaders import RecursiveUrlLoader

loader = RecursiveUrlLoader("https://www.4399.com/", max_depth=1)
documents = loader.load()


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents)


In [26]:
# pip install -qU langchain-google-genai

In [27]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [28]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [29]:
document_ids = vector_store.add_documents(documents=all_splits)

In [31]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1, "fetch_k": 2, "lambda_mult": 0.5},
)
retriever.invoke("something")

[Document(id='b21b270b-2507-437a-aceb-0bd9610a5928', metadata={'source': 'https://www.4399.com/', 'content_type': 'text/html', 'title': '小游戏,4399小游戏,小游戏大全,双人小游戏大全 - www.4399.com', 'description': '4399小游戏大全包含连连看 ,连连看小游戏大全,双人小游戏大全,H5在线小游戏,4399洛克王国,4399赛尔号,4399奥拉星,4399奥比岛,4399弹弹堂,4399单人小游戏,奥比岛小游戏,造梦西游online,造梦无双等最新小游戏。', 'language': None}, page_content='<li><a href="/special/154.htm"><img lzimg="1" lz_src="//imga4.5054399.com/upload_pic/2020/12/4/4399_10313133912.jpg">美发</a></li>\n                \t\t\t\t<li><a href="/special/182.htm"><img lzimg="1" lz_src="//imga4.5054399.com/upload_pic/2020/12/4/4399_10310035640.jpg">朵拉</a></li>\n                \t\t\t\t<li><a href="/special/7.htm"><img lzimg="1" lz_src="//imga5.5054399.com/upload_pic/2020/12/4/4399_10302261117.jpg">做饭</a></li>\n                \t\t\t\t<li><a href="/special/109.htm"><img lzimg="1" lz_src="//imga3.5054399.com/upload_pic/2020/12/4/4399_13561494453.jpg">宠物</a></li>\n                \t\t\t\t<li><a href="/special/23.htm"><im

In [32]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [33]:
# 参考文档：
# https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html


In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub


retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")


combine_docs_chain = create_stuff_documents_chain(
    model, retrieval_qa_chat_prompt
)

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

retrieval_chain.invoke({"input": "..."})

{'input': '...',
 'context': [Document(id='6c0c14dc-1b9b-4f00-b09e-335daf07ff5f', metadata={'source': 'https://www.4399.com/', 'content_type': 'text/html', 'title': '小游戏,4399小游戏,小游戏大全,双人小游戏大全 - www.4399.com', 'description': '4399小游戏大全包含连连看 ,连连看小游戏大全,双人小游戏大全,H5在线小游戏,4399洛克王国,4399赛尔号,4399奥拉星,4399奥比岛,4399弹弹堂,4399单人小游戏,奥比岛小游戏,造梦西游online,造梦无双等最新小游戏。', 'language': None}, page_content='<div class="more">\n\t\t\t\t\t\t\t<a class="game" href="https://my.4399.com/yxsgs/"><img lz_src="//imga5.5054399.com/upload_pic/2025/7/11/4399_14404962067.jpg" alt=""></a>\n\t\t\t\t\t\t\t<div class="names">\n\t\t\t\t\t\t\t\t<a class="name" href="https://my.4399.com/yxsgs/">4399三国杀</a>\n\t\t\t\t\t\t\t\t<a class="play" href="https://my.4399.com/yxsgs/">开始游戏</a>\n\t\t\t\t\t\t\t</div>\n\t\t\t\t\t\t</div>\n\t\t\t\t\t</li>\n\t\t\t\t\t\t\t\t\t<li class="">\n\t\t\t\t\t\t<em class="bg10">10</em>\n\t\t\t\t\t\t<a class="btn-name" href="https://my.4399.com/yxmsdzls/">美食大战老鼠</a>\n\t\t\t\t\t\t<div class="more">\n\t\t\t\t\t\t

In [35]:
query = "你能检索到什么？"
response = retrieval_chain.invoke({"input": query})
print(response["answer"])


根据提供的内容，我可以检索到以下信息：

*   **网站名称：** 4399小游戏
*   **网站提供的游戏类型：** 连连看小游戏，双人小游戏，H5在线小游戏，洛克王国，赛尔号，奥拉星，奥比岛，弹弹堂，单人小游戏，造梦西游online，造梦无双等。
*   **网站关键词：** 小游戏, 4399小游戏, 在线小游戏, 双人小游戏, 小游戏大全, 双人小游戏大全
*   **网站描述：** 4399小游戏大全包含连连看 ,连连看小游戏大全,双人小游戏大全,H5在线小游戏,4399洛克王国,4399赛尔号,4399奥拉星,4399奥比岛,4399弹弹堂,4399单人小游戏,奥比岛小游戏,造梦西游online,造梦无双等最新小游戏。


In [36]:
query = "你能检索到什么？"
response = model.invoke(query)
print(response)


content='我被设计用来访问和处理来自各种来源的信息，这使我能够检索并提供各种各样的信息，包括：\n\n**1. 文本信息：**\n\n*   **事实性信息：** 我可以查找和总结事实、日期、统计数据、定义等。\n*   **新闻和事件：** 我可以提供最新的新闻、事件和趋势的摘要。\n*   **文章和文档：** 我可以访问和处理各种文章、博客帖子、研究论文、书籍和其他文档。\n*   **定义和解释：** 我可以提供单词、概念和术语的定义和解释。\n*   **指令和指南：** 我可以提供如何做事的指导、食谱、教程等。\n*   **文学作品:** 我可以访问和处理诗歌、小说、戏剧等文学作品。\n\n**2. 代码：**\n\n*   **编程语言代码：** 我可以理解和生成多种编程语言的代码，例如Python、Java、JavaScript、C++等。\n*   **算法和数据结构：** 我可以提供算法和数据结构的解释和实现。\n*   **代码示例：** 我可以提供代码示例来解决特定的编程问题。\n\n**3. 多媒体信息 (通过其他工具和 APIs):**\n\n*   **图像:**  虽然我不能直接 *显示* 图像，但我可以描述图像内容、识别图像中的物体、或者提供图像搜索链接。\n*   **音频:**  类似图像，我不能直接播放音频，但我可以描述音频内容或者提供音频搜索链接。\n*   **视频:**  同样，我可以描述视频内容或者提供视频搜索链接。\n\n**4. 常识推理和逻辑推断：**\n\n*   **回答复杂问题：** 我可以根据已有的知识和信息回答复杂的问题，并进行推理。\n*   **解决逻辑问题：** 我可以解决逻辑问题，例如谜题和推理题。\n*   **生成创意文本格式：** 我可以生成不同创意文本格式的文本内容，例如诗歌、代码、剧本、音乐作品、电子邮件、信件等。\n\n**总而言之，我能检索的信息范围非常广泛，涵盖了人类知识的许多领域。  我的能力还在不断发展，未来会更加强大。**\n\n**我不能检索的信息包括：**\n\n*   **个人身份信息（PII）除非你明确告诉我并要求我处理，我不会主动搜索或存储你的个人信息。**  这包括你的姓名、地址、电话号码、电子邮件地址、社会安全号码等。\n*   **实时信息：** 